In [ ]:
# 🧠 Clasificación de Cáncer de Mama con CNNs
# Proyecto: Breast Histopathology Images (IDC)

# ---------------------------------------------
# ⚙️ Setup inicial en Google Colab
# ---------------------------------------------

# Si es tu primera vez:
# !apt-get install git

# 🔁 Clonar repositorio del curso
!git clone https://github.com/solivare/DeepNeuralNetworkUSS.git

# Cambiar al directorio del proyecto
%cd DeepNeuralNetworkUSS/Examples/BreastCancer/

# Instalar dependencias
!pip install -r requirements.txt

In [ ]:
# ⚙️ Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📁 Verificar contenido 
!ls "/content/drive/MyDrive/breast_images_prepared"

In [ ]:
# ---------------------------------------------
# 🛠️ Ajuste de rutas de importación
# ---------------------------------------------

import os
import sys

src_path = os.path.abspath("src")
if src_path not in sys.path:
    sys.path.append(src_path)

In [ ]:
# ---------------------------------------------
# 🧼 Preprocesamiento de los datos
# ---------------------------------------------

#%cd src
#!python preprocess.py
#%cd ..

In [ ]:
# ---------------------------------------------
# 🧼 Renombrar directorio de datos preprocesados
# ---------------------------------------------

# 🔁 Sobrescribir la ruta de los datos con la de Drive
config = train.load_config("config.yaml")
config["paths"]["prepared_data"] = "/content/drive/MyDrive/breast_images_prepared"

In [ ]:
# ---------------------------------------------
# 🧠 Entrenamiento del modelo
# ---------------------------------------------

import train
import model
import utils
import evaluate as ev

#config = train.load_config("config.yaml")
#cnn_model, history = train.train_model(config)

#utils.plot_training_history(history, model_name="CNN Breast Cancer")
#utils.save_model(cnn_model, path=config["paths"]["model"])

# 📊 Cargar datos y entrenar modelo
train_gen, val_gen = train.get_data_generators(config)

cnn_model = model.build_model(config)

history = cnn_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=config["training"]["epochs"]
)

utils.save_model(cnn_model, config["paths"]["model"])

In [ ]:
# ---------------------------------------------
# 📈 Evaluación del modelo
# ---------------------------------------------

from tensorflow.keras.models import load_model

#cnn_model = load_model(config["paths"]["model"])

# Obtener datos de validación
#val_gen = train.get_data_generators(config)[1]
#y_true = val_gen.classes
#y_proba = cnn_model.predict(val_gen).flatten()
#threshold = config.get("evaluation", {}).get("threshold", 0.5)
#y_pred = (y_proba >= threshold).astype(int)

# Métricas y visualizaciones
#ev.plot_confusion_matrix(y_true, y_pred)
#ev.plot_roc_curve(y_true, y_proba)
#ev.plot_prediction_distribution(y_true, y_proba)
#ev.plot_f1_vs_threshold(y_true, y_proba, modelo="CNN")

# 📈 Evaluación del modelo
y_true = val_gen.classes
y_scores = cnn_model.predict(val_gen).flatten()
y_pred = ev.predict_with_threshold(y_scores, threshold=config["evaluation"]["threshold"])

ev.plot_confusion_matrix(y_true, y_pred)
ev.plot_roc_curve(y_true, y_scores)
ev.plot_prediction_distribution(y_true, y_scores)
ev.plot_f1_vs_threshold(y_true, y_scores)


In [ ]:
# ---------------------------------------------
# 🔍 Análisis de errores y Grad-CAM
# ---------------------------------------------

# Mostrar imágenes mal clasificadas
ev.show_misclassified_images(val_gen, cnn_model)

In [ ]:
# ---------------------------------------------
# ✅ Reflexión final
# ---------------------------------------------
# - ¿El modelo muestra overfitting? ¿Cómo lo ves en las curvas?
# - ¿Qué tan bien discrimina entre clases? ¿AUC > 0.85?
# - ¿El modelo predice con alta confianza o con muchas dudas?
# - ¿Qué otras técnicas podrías aplicar para mejorar resultados?